In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from typing import Dict, Tuple
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from model import arreturn, features_label, flawed_model, kfold_cv

In [ ]:
# -----------------------------
# simulate returns & quick plot
# -----------------------------
seed = 76
rng = np.random.default_rng(seed)
returns, alphas = arreturn(rng)   # alphas shape = (n, 2)

# Plot returns and alpha coefficients
plt.figure(figsize=(12,4))
plt.plot(returns, label='returns')
plt.plot(alphas[:,0], label='alpha1')   # first AR coefficient over time
plt.plot(alphas[:,1], label='alpha2')   # second AR coefficient over time
plt.title('Simulated returns with regime coefficients')
plt.legend()
plt.show()

# -----------------------------
# Prepare training data (in-sample)
# -----------------------------
returns_train = returns[0:404]   # using an early slice for training experiments
X, Y = features_label(returns_train)

# Train a model on the whole training block and report in-sample loss
full_model = flawed_model(X.shape[1])

history = full_model.fit(
    X, Y,
    epochs=100,
    batch_size=32,
    verbose=0
)

# Plot the convergence in training error
plt.figure(figsize=(12,4))
plt.plot(history.history['loss'], marker = 'D', ms = 3.0)
plt.title('training loss after each epoch')
plt.show()

print('final training error:', history.history['loss'][-1])


# -----------------------------
# K-fold cross-validation (contiguous blocks)
# -----------------------------

val_losses = kfold_cv(X, Y)

print("\nMean CV loss:", np.mean(val_losses))
print("Std CV loss:", np.std(val_losses))


# -----------------------------
# True out-of-sample test (far future slice)
# -----------------------------
returns_test = returns[3000:]               # a future block far away
X_test, Y_test = features_label(returns_test)
test_loss = full_model.evaluate(X_test, Y_test, verbose=0)   # using full model trained on entire returns_train

print('test loss:', test_loss)
